In [383]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [384]:
#load all ti matches
matches_df = pd.read_json('all_matches.json')
matches_df['match_id'].count()



76

In [385]:
# determine winner based on match_id
def get_winner(matchid):
    match_data = matches_df[matches_df['match_id']==matchid]
    radiant_win = match_data['radiant_win'].to_string(index=False)
    winner = 1
    if radiant_win == 'True':
        winner = 0
    return winner



In [386]:
#define heroes df
heroes_df = pd.read_json('heroes.json')
heroes_df = heroes_df[['id','localized_name']]
heroes_df.rename(columns={"localized_name":"hero_name"},inplace=True)
heroes_df.head()


,id,hero_name
0,1,Anti-Mage
1,2,Axe
2,3,Bane
3,4,Bloodseeker
4,5,Crystal Maiden


In [387]:
# define players df
ti_teams_df= pd.read_json('teams.json')
ti_teams = list(ti_teams_df['team_id'])
players_df = pd.read_json('proPlayers.json')
players_df = players_df[['account_id','personaname','name','team_id','team_name','team_tag']]
players_df = players_df[players_df['team_id'].isin(ti_teams)]
players_df.head()

,account_id,personaname,name,team_id,team_name,team_tag
17,10366616,Sneyking,Sneyking,8291895,Tundra Esports,Tundra
30,16497807,tOfu,tOfu,8599101,Gaimin Gladiators,GG
52,25907144,Let's Go,Cr1t-,39,Evil Geniuses,EG
83,38628747,Verissimus,MoonMeander,8260983,TSM FTX,TSM FTX
91,40547474,Aui_2000,Aui_2000,8291895,Tundra Esports,Tundra


In [388]:
#create pickbans df
picks_bans = matches_df.set_index('match_id').to_dict()['picks_bans']
picks = []
bans = []
for i in picks_bans:
    winner = get_winner(i)
    for j in picks_bans[i]:
        if j['is_pick']:
            pick = {}
            pick['match_id'] = i
            pick['order'] = j['order']
            pick['hero_id'] = j['hero_id']
            pick['team'] = j['team']
            if winner == j['team']:
                pick['win'] = 1
            else:
                pick['win'] = 0
            picks.append(pick)
        elif not j['is_pick']:
            ban = {}
            ban['match_id'] = i
            ban['order'] = j['order']
            ban['hero_id'] = j['hero_id']
            ban['team'] = j['team']
            bans.append(ban)

In [389]:
picks_df = pd.DataFrame(picks)
heroes_df['pick_count'] = heroes_df['id'].map(picks_df['hero_id'].value_counts())
wins = picks_df[['hero_id','win']].groupby(['hero_id']).sum()
wins = wins.to_dict()
heroes_df['wins'] = heroes_df['id'].map(wins['win'])
heroes_df['winrate'] = round(heroes_df['wins'] / heroes_df['pick_count'] * 100,2)

bans_df = pd.DataFrame(bans)
heroes_df['ban_count'] = heroes_df['id'].map(bans_df['hero_id'].value_counts())


heroes_df.head()

,id,hero_name,pick_count,wins,winrate,ban_count
0,1,Anti-Mage,1.0,1.0,100.0,1.0
1,2,Axe,NaN,NaN,NaN,2.0
2,3,Bane,2.0,1.0,50.0,NaN
3,4,Bloodseeker,2.0,0.0,0.0,24.0
4,5,Crystal Maiden,12.0,9.0,75.0,1.0


In [390]:
#create hero stats
players = matches_df.set_index('match_id').to_dict()['players']
hero_stats = []
for i in players:
    for j in players[i]:
        hero_stat = {}
        hero_stat['match_id'] = j['match_id']
        hero_stat['account_id'] = j['account_id']
        hero_stat['hero_id'] = j['hero_id']
        hero_stat['kills'] = j['kills']
        hero_stat['deaths'] = j['deaths']
        hero_stat['assists'] = j['assists']
        hero_stat['last_hits'] = j['last_hits']
        hero_stat['xp_per_min'] = j['xp_per_min']
        hero_stat['gold_per_min'] = j['gold_per_min']
        hero_stats.append(hero_stat)
hero_stats_df = pd.DataFrame(hero_stats)
players_teams = players_df.set_index('account_id').to_dict()['team_id']
hero_stats_df['team_id'] = hero_stats_df['account_id'].map(players_teams) 

hero_stats_df.head()

,match_id,account_id,hero_id,kills,deaths,assists,last_hits,xp_per_min,gold_per_min,team_id
0,6806663153,86799300,102,0,4,7,51,361,222,7390454
1,6806663153,221666230,52,6,4,5,300,578,526,7390454
2,6806663153,87063175,137,6,4,6,178,462,378,7390454
3,6806663153,108452107,10,2,0,1,331,647,535,7390454
4,6806663153,86726887,128,2,6,10,42,301,187,7390454


In [391]:
hero_avg_stats = hero_stats_df.groupby('hero_id').mean()
hero_max_stats = hero_stats_df.groupby('hero_id').max()
hero_min_stats = hero_stats_df.groupby('hero_id').min()

heroes_df['avg_kills'] = heroes_df['id'].map(hero_avg_stats['kills'])

heroes_df['max_kills'] = heroes_df['id'].map(hero_max_stats['kills'])

heroes_df['avg_deaths'] = heroes_df['id'].map(hero_avg_stats['deaths'])

heroes_df['min_deaths'] = heroes_df['id'].map(hero_min_stats['deaths'])

heroes_df['avg_assists'] = heroes_df['id'].map(hero_avg_stats['assists'])

heroes_df['max_assists'] = heroes_df['id'].map(hero_max_stats['assists'])

heroes_df['avg_last_hits'] = heroes_df['id'].map(hero_avg_stats['last_hits'])

heroes_df['max_last_hits'] = heroes_df['id'].map(hero_max_stats['last_hits'])

heroes_df['avg_xp_per_min'] = heroes_df['id'].map(hero_avg_stats['xp_per_min'])

heroes_df['avg_gold_per_min'] = heroes_df['id'].map(hero_avg_stats['gold_per_min'])

heroes_df.head()

,id,hero_name,pick_count,wins,winrate,ban_count,avg_kills,max_kills,avg_deaths,min_deaths,avg_assists,max_assists,avg_last_hits,max_last_hits,avg_xp_per_min,avg_gold_per_min
0,1,Anti-Mage,1.0,1.0,100.0,1.0,7.000000,7.0,6.00,6.0,13.0,13.0,471.000000,471.0,927.000000,700.000000
1,2,Axe,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Bane,2.0,1.0,50.0,NaN,1.000000,2.0,4.00,2.0,6.5,11.0,31.000000,41.0,337.000000,232.000000
3,4,Bloodseeker,2.0,0.0,0.0,24.0,9.000000,14.0,4.00,4.0,10.0,12.0,396.500000,466.0,698.000000,601.500000
4,5,Crystal Maiden,12.0,9.0,75.0,1.0,2.583333,5.0,5.75,2.0,15.5,24.0,65.916667,116.0,409.166667,303.333333


In [392]:
heroes_5_df = heroes_df[heroes_df['pick_count']>=5]

In [393]:
player_avg_stats = hero_stats_df.groupby('account_id').mean()
player_max_stats = hero_stats_df.groupby('account_id').max()
player_min_stats = hero_stats_df.groupby('account_id').min()

players_df['avg_kills'] = players_df['account_id'].map(player_avg_stats['kills'])

players_df['max_kills'] = players_df['account_id'].map(player_max_stats['kills'])

players_df['avg_deaths'] = players_df['account_id'].map(player_avg_stats['deaths'])

players_df['min_deaths'] = players_df['account_id'].map(player_min_stats['deaths'])

players_df['avg_assists'] = players_df['account_id'].map(player_avg_stats['assists'])

players_df['max_assists'] = players_df['account_id'].map(player_max_stats['assists'])

players_df['avg_last_hits'] = players_df['account_id'].map(player_avg_stats['last_hits'])

players_df['max_last_hits'] = players_df['account_id'].map(player_max_stats['last_hits'])

players_df['avg_xp_per_min'] = players_df['account_id'].map(player_avg_stats['xp_per_min'])

players_df['avg_gold_per_min'] = players_df['account_id'].map(player_avg_stats['gold_per_min'])

players_df['max_gold_per_min'] = players_df['account_id'].map(player_max_stats['gold_per_min'])

player_unique_heroes = hero_stats_df[['account_id','hero_id']].groupby('account_id').nunique()
player_unique_heroes = player_unique_heroes.to_dict()
players_df['unique_heroes'] = players_df['account_id'].map(player_unique_heroes['hero_id'])

players_df.head()

,account_id,personaname,name,team_id,team_name,team_tag,avg_kills,max_kills,avg_deaths,min_deaths,avg_assists,max_assists,avg_last_hits,max_last_hits,avg_xp_per_min,avg_gold_per_min,max_gold_per_min,unique_heroes
17,10366616,Sneyking,Sneyking,8291895,Tundra Esports,Tundra,3.000000,6.0,7.714286,2.0,15.000000,29.0,69.285714,125.0,445.142857,293.857143,363.0,5.0
30,16497807,tOfu,tOfu,8599101,Gaimin Gladiators,GG,3.125000,7.0,5.125000,2.0,15.625000,38.0,81.500000,105.0,470.500000,310.000000,395.0,6.0
52,25907144,Let's Go,Cr1t-,39,Evil Geniuses,EG,5.125000,12.0,4.875000,0.0,15.625000,23.0,56.500000,82.0,477.000000,310.625000,376.0,5.0
83,38628747,Verissimus,MoonMeander,8260983,TSM FTX,TSM FTX,3.285714,15.0,7.000000,2.0,9.285714,19.0,70.142857,109.0,395.285714,277.571429,423.0,5.0
91,40547474,Aui_2000,Aui_2000,8291895,Tundra Esports,Tundra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [394]:
ti_teams_df = ti_teams_df[['team_id','name','tag']]
teams_stats_df = hero_stats_df.groupby(['match_id','team_id']).sum()
team_avg_stats_df = teams_stats_df.groupby('team_id').mean()
team_max_stats_df = teams_stats_df.groupby('team_id').max()
team_min_stats_df = teams_stats_df.groupby('team_id').min()
team_avg_kills

ti_teams_df['avg_kills'] = ti_teams_df['team_id'].map(team_avg_stats_df['kills'])
ti_teams_df['avg_deaths'] = ti_teams_df['team_id'].map(team_avg_stats_df['deaths'])
ti_teams_df['avg_assists'] = ti_teams_df['team_id'].map(team_avg_stats_df['assists'])
ti_teams_df['max_kills'] = ti_teams_df['team_id'].map(team_max_stats_df['kills'])
ti_teams_df['min_deaths'] = ti_teams_df['team_id'].map(team_min_stats_df['deaths'])
ti_teams_df['max_assists'] = ti_teams_df['team_id'].map(team_max_stats_df['assists'])
ti_teams_df.head()

,team_id,name,tag,avg_kills,avg_deaths,avg_assists,max_kills,min_deaths,max_assists
0,8597976,Talon,TLN,21.857143,27.571429,43.857143,46,10,81
1,7732977,BOOM Esports,BOOM,26.000000,23.750000,61.000000,52,6,132
2,7390454,Soniqs,SQ,20.125000,22.250000,44.250000,40,9,99
3,2163,Team Liquid,Liquid,27.000000,24.750000,63.875000,42,10,104
4,8260983,TSM FTX,TSM FTX,17.714286,26.714286,37.857143,37,12,86


In [449]:
def get_matchwinner(row):
    if row['radiant_win'] == True:
        winner = 'radiant_team_id'
    else:
        winner = 'dire_team_id'
    return row[winner]

def convert(row,column):
    seconds = row[column]
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
     
    return "%d:%02d:%02d" % (hour, minutes, seconds)

ti_matches_df = matches_df[['match_id','duration','radiant_win','radiant_score','radiant_team_id','dire_score','dire_team_id']]
ti_matches_df['total_score'] = ti_matches_df['radiant_score']+ti_matches_df['dire_score']

ti_matches_df['team_winner'] = ti_matches_df.apply(lambda x: get_matchwinner(x), axis=1)
ti_matches_df['team_winner'] = ti_matches_df['team_winner'].map(ti_teams_df.set_index('team_id').to_dict()['name'])
ti_matches_df['duration_minutes'] = ti_matches_df.apply(lambda x: convert(x,'duration'), axis=1)
ti_matches_df.head()

,match_id,duration,radiant_win,radiant_score,radiant_team_id,dire_score,dire_team_id,total_score,team_winner,duration_minutes
0,6806663153,2178,False,16,7390454,18,15,34,PSG.LGD,0:36:18
1,6806047576,2971,False,23,7732977,24,2586976,47,OG,0:49:31
2,6806638898,3098,False,19,7732977,28,6209804,47,Royal Never Give Up,0:51:38
3,6806648571,2250,True,33,8131728,14,8599101,47,Hokori,0:37:30
4,6806662696,1455,False,5,2586976,34,39,39,Evil Geniuses,0:24:15


In [450]:
radiant_teams = ti_matches_df[['match_id','radiant_team_id','duration']].rename(columns={"radiant_team_id":"team_id"})
dire_teams = ti_matches_df[['match_id','dire_team_id','duration']].rename(columns={"dire_team_id":"team_id"})
ti_teams_matches = pd.concat([radiant_teams,dire_teams],ignore_index=True)
ti_teams_df['avg_length'] = ti_teams_df['team_id'].map(ti_teams_matches.groupby('team_id').mean()['duration'])
ti_teams_df['avg_length_minutes'] = ti_teams_df.apply(lambda x: convert(x,'avg_length'), axis=1)

# Hero with the most picked:

In [396]:
heroes_df[heroes_df['pick_count']==heroes_df['pick_count'].max()][['hero_name','pick_count']]

,hero_name,pick_count
121,Marci,30.0


# Hero with the most banned:

In [397]:
heroes_df[heroes_df['ban_count']==heroes_df['ban_count'].max()][['hero_name','ban_count']]

,hero_name,ban_count
31,Enigma,50.0


# Hero with the highest win rate:

In [398]:
heroes_5_df[heroes_5_df['winrate']==heroes_5_df['winrate'].max()][['hero_name','winrate']]

,hero_name,winrate
62,Jakiro,83.33


# Hero with the highest average kill:

In [399]:
heroes_5_df[heroes_5_df['avg_kills']==heroes_5_df['avg_kills'].max()][['hero_name','avg_kills']]



,hero_name,avg_kills
33,Sniper,10.428571


# Hero with the highest average assist:

In [400]:
heroes_5_df[heroes_5_df['avg_assists']==heroes_5_df['avg_assists'].max()][['hero_name','avg_assists']]

,hero_name,avg_assists
73,Silencer,18.066667


# Hero with the lowest average death:

In [401]:
heroes_5_df[heroes_5_df['avg_deaths']==heroes_5_df['avg_deaths'].min()][['hero_name','avg_deaths']]

,hero_name,avg_deaths
44,Templar Assassin,2.5


# Hero with the highest average last hit:

In [402]:
heroes_5_df[heroes_5_df['avg_last_hits']==heroes_5_df['avg_last_hits'].max()][['hero_name','avg_last_hits']]

,hero_name,avg_last_hits
87,Naga Siren,562.1


# Hero with the highest average xp per minute:

In [403]:
heroes_5_df[heroes_5_df['avg_xp_per_min']==heroes_5_df['avg_xp_per_min'].max()][['hero_name','avg_xp_per_min']]

,hero_name,avg_xp_per_min
44,Templar Assassin,773.0


# Hero with the most kills in a game:

In [404]:
heroes_5_df[heroes_5_df['max_kills']==heroes_5_df['max_kills'].max()][['hero_name','max_kills']]

,hero_name,max_kills
50,Leshrac,22.0


# Hero with the most last hits in a game:

In [405]:
heroes_5_df[heroes_5_df['max_last_hits']==heroes_5_df['max_last_hits'].max()][['hero_name','max_last_hits']]

,hero_name,max_last_hits
33,Sniper,1058.0


# Player with the highest kill average:

In [406]:
players_df[players_df['avg_kills']==players_df['avg_kills'].max()][['name','avg_kills']]

,name,avg_kills
723,Somnus,10.125


# Player with the highest assist average:

In [407]:
players_df[players_df['avg_assists']==players_df['avg_assists'].max()][['name','avg_assists']]

,name,avg_assists
1267,kaka,17.125


# Player with the lowest death average:

In [408]:
players_df[players_df['avg_deaths']==players_df['avg_deaths'].min()][['name','avg_deaths']]

,name,avg_deaths
1340,Monet,1.428571


# Player with the highest last hit average:

In [409]:
players_df[players_df['avg_last_hits']==players_df['avg_last_hits'].max()][['name','avg_last_hits']]

,name,avg_last_hits
2454,JACKBOYS,526.125


# Player with the highest GPM average:

In [410]:
players_df[players_df['avg_gold_per_min']==players_df['avg_gold_per_min'].max()][['name','avg_gold_per_min']]

,name,avg_gold_per_min
2284,Pakazs,701.285714


# Player with the most kills:

In [411]:
players_df[players_df['max_kills']==players_df['max_kills'].max()][['name','max_kills']]

,name,max_kills
723,Somnus,22.0


# Player with the most assists:

In [412]:
players_df[players_df['max_assists']==players_df['max_assists'].max()][['name','max_assists']]

,name,max_assists
1418,TIMS,39.0


# Player with the most last hits:

In [413]:
players_df[players_df['max_last_hits']==players_df['max_last_hits'].max()][['name','max_last_hits']]

,name,max_last_hits
2454,JACKBOYS,1058.0


# Player with the most GPM:

In [414]:
players_df[players_df['max_gold_per_min']==players_df['max_gold_per_min'].max()][['name','max_gold_per_min']]

,name,max_gold_per_min
909,dyrachyo,995.0


# Player with the most different heroes:

In [415]:
players_df[players_df['unique_heroes']==players_df['unique_heroes'].max()][['name','unique_heroes']]

,name,unique_heroes
211,zai,8.0
1075,Gard1ck,8.0
2669,萧瑟,8.0


# Team with the most kills in a game:

In [416]:
ti_teams_df[ti_teams_df['max_kills']==ti_teams_df['max_kills'].max()][['name','max_kills']]

,name,max_kills
7,Gaimin Gladiators,54


# Team with the highest kill average:

In [417]:
ti_teams_df[ti_teams_df['avg_kills']==ti_teams_df['avg_kills'].max()][['name','avg_kills']]

,name,avg_kills
13,Evil Geniuses,29.5


# Team with the least deaths in a game:

In [418]:
ti_teams_df[ti_teams_df['min_deaths']==ti_teams_df['min_deaths'].min()][['name','min_deaths']]

,name,min_deaths
15,Team Aster,3


# Team with the most assists in a game:

In [419]:
ti_teams_df[ti_teams_df['max_assists']==ti_teams_df['max_assists'].max()][['name','max_assists']]

,name,max_assists
1,BOOM Esports,132
7,Gaimin Gladiators,132
14,Fnatic,132


# Team who wins the longest game:

In [420]:
ti_matches_df[ti_matches_df['duration']==ti_matches_df['duration'].max()][['team_winner','duration_minutes']]

,team_winner,duration_minutes
49,Royal Never Give Up,1:10:29


# Team who wins the shortest game:

In [421]:
ti_matches_df[ti_matches_df['duration']==ti_matches_df['duration'].min()][['team_winner','duration_minutes']]



,team_winner,duration_minutes
14,Team Secret,0:19:30


# Team with the highest length average:

In [451]:
ti_teams_df[ti_teams_df['avg_length']==ti_teams_df['avg_length'].max()][['name','avg_length_minutes']]

,name,avg_length_minutes
1,BOOM Esports,0:47:07
